In [35]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt

In order to manage our algorithm's computations effectively, we have introduced an `Ant` class. This class is designed to encapsulate all the attributes for each ant's individual solution within the algorithm's context. Detailed below are the key attributes and methods this class has:

### Attributes

- **U (Universe):** Represents the complete set of elements available for forming solutions. It defines the total space from which sets are selected.
- **S (Sets):** A collection of all sets available for selection.
- **C (Cost):** Associated with each set in `S`, this attribute represents the cost of including a particular set in the solution. It’s utilized to calculate the overall solution cost.
- **Solution (sol):** This attribute is a list containing indexes of sets in `S` that comprise the ant's final solution. It effectively represents which sets from `S` have been chosen.
- **uncovered:** A set containing elements from the universe (`U`) that are yet to be covered by the chosen sets in the final solution. It helps in identifying the elements still missing from the solution.
- **cost:** Denotes the overall cost of the ant's solution, derived by summing up the costs of all selected sets within the solution.

### Methods

1. **setUncovered():** This method operates by deducting elements of the selected sets (as part of the solution) from the universe (`U`). The resulting set captures elements that are still uncovered, i.e., elements of `U` not included in any of the selected sets. It refreshes the `uncovered` attribute with these elements, thus identifying what's missing from the current solution.

2. **calculateCost():** By iterating through the indexes listed in the `sol` attribute (which indicates the sets included in the solution), this method accumulates the costs associated with each selected set. Summing these costs provides the total cost of the ant's solution, which is then stored in the Cost attribute.


In [36]:
class Ant():
    def __init__(self, universe, sets, costs, solution):
        self.U = universe
        self.S = sets
        self.C = costs
        self.sol = solution
        self.uncovered = self.setUncovered()
        self.cost = self.calculateCost()
        
    def setUncovered(self):
        uncov = set(self.U)
        if not self.sol:
            return uncov
        for colx in self.sol:
            uncov -= set(self.S[colx])
        
        self.uncovered = uncov
        return uncov
    
    def calculateCost(self):
        if not self.sol:
            self.cost = 0
            return 0
        cost = 0
        for j in self.sol:
            cost += self.C[j]
        self.cost = cost
        return cost

In [37]:
# the function to get the input and save it's components to U, S and C
def getInput(file_path):
    with open(file_path, 'r') as file:
        n, m = map(int, file.readline().strip().split())

        costs = []
        while len(costs) < m:
            costs += list(map(int, file.readline().strip().split()))

        U = list(range(n))
        S = [[] for _ in range(m)]
        for i in range(n):
            nn = int(file.readline())
            cols = []
            while len(cols) < nn:
                cols += list(map(int, file.readline().strip().split()))
            for j in cols:
                S[j - 1].append(i)
    
    print('Length of 1D array Cost:', len(costs))    
    print('Length of 1D array U (universe):', len(U))
    print('Length of 2D array S:', len(S))    

    return costs, U, S

In alignment with the framework detailed in the paper titled "[Ant Colonies for the Set Covering Problem](https://www.researchgate.net/publication/245585705_Ant_Colonies_for_the_Set_Covering_Problem)," we've developed a class named `MMAS` to encapsulate our version of the Min-Max Ant Colony System. This class is architected to facilitate our specific needs in solving the set covering problem through a series of well-defined steps:

**Constructor: `__init__()`**

The constructor initializes the class with various attributes, each playing a pivotal role in different segments of our algorithm. These attributes include parameters like the number of ants, number of iterations, and initial pheromone levels, among others. Descriptive comments within the codebase clarify the utilization context for each attribute.


**`runMMAS()`**

This primary method orchestrates the execution of the Min-Max Ant System algorithm by performing the following steps across each iteration:

1. **Diversification:** Drawing upon the diversification algorithm discussed in the referenced paper, an initial assortment of column indexes is assigned to each ant. This step lays the groundwork for constructing the actual solution, ensuring a broad exploration of the solution space.

2. **constructSolution:** Here, we meticulously build the solution, ensuring comprehensive coverage. The goal is to construct a solution where no element remains uncovered, fostering an environment for optimal solution discovery.

3. **eliminateRedundants:** In an effort to streamline the solution, this step involves identifying and removing redundant columns (each representing a set). Redundancies are detected when a set is a subset of another. Priority is given to removing sets with higher associated costs, thereby contributing to a more cost-effective solution.

4. **updateIterationOptimum:** Once an ant's solution is formulated and refined, it becomes crucial to record the iteration's best outcomes in terms of cost, solution, and the ant responsible. This benchmarking is instrumental for visualizing progress and guiding the algorithm towards achieving optimal results.

5. **updatePheromoneIB:** This method applies iteration-based updates to the pheromone trails, as detailed in the original paper. It's essential for steering subsequent solution searches. The method also adjusts $\tau_{max}$ and $\tau_{min}$ values, which are critical in the context of a Min-Max algorithm, ensuring the pheromone concentration remains within designated bounds.

Upon completion of these steps for all ants, the iteration concludes with the identification of the best solution derived thus far. A local search mechanism is employed at this juncture to further refine the discovered solution, thereby amplifying the algorithm's efficacy in unearthing superior results.

**`diversification(self)`**

This method strategically employs the input parameter `nc_max` to determine the initial number of columns to be included in the ant’s solution. To avoid inadvertently selecting columns with high associated costs from the onset, the approach involves randomly selecting $k$ elements from the first quarter of the set collection. This segment is chosen based on the fact that it comprises sets with lower costs, thereby facilitating a cost-effective start to the solution construction process.

$***$ ***Challenge*** $***$

The main challenge I faced was the tendency for random selection to include very high-cost sets in the solution set. While these sets can potentially be removed during the elimination or local search processes, the effectiveness of these mechanisms is not guaranteed. The elimination process relies on identifying redundant sets, which might not always be chosen, and the local search process still incorporates an element of randomness, making it possible for high-cost sets to persist in the final solution.

To overcome this issue, I chose to focus on the lower-cost sets, aiming to build a more cost-effective initial solution. However, to avoid gravitating towards a local optimum by only selecting the cheapest options, I lowered the `nc_max` parameter. This change limited the number of low-cost sets added to the solution, striking a balance between cost-efficiency and the diversity necessary to escape local optima.

**`constructSolution(self, ant)`**

The process continues by selecting sets to add to an ant’s solution until all elements are covered. To achieve this, we use the `calcProbability()` function to determine the ‘most probable’ set for inclusion. This function assigns a probability to each set, reflecting its likelihood of being chosen next. A set is then selected based on these probabilities and added to the ant’s solution, ensuring that every element is eventually covered. the following functions are used in solution construction:

- **`calcProbability(self, ant)`**
    This method computes the probability ($P$) of selecting each set for inclusion in the ant's solution, based on both the pheromone trails ($\tau$) and the heuristic values ($\eta$), which are calculated by `heuristicValue()`. The calculation also incorporates two parameters, $\alpha$ and $\beta$, which balance the influence of pheromone intensity ($\tau$) and heuristic desirability ($\eta$) on the decision process. The steps are as follows:
    
    1. Retrieve the list of heuristic values ($\eta$) for the current state of the ant's solution using `heuristicValue(ant)`.
    2. Calculate the denominator of the probability formula, which is the sum of the products of pheromone intensity ($\tau$) and heuristic value ($\eta$) for each set, each raised to the power of their respective weights ($\alpha$ for $\tau$ and $\beta$ for $\eta$). This sum aggregates the weighted contributions of all sets based on their current attractiveness and pheromone levels.
    3. Initialize an empty list $P$ to store probabilities for each set.
    4. Iterate over the sets, calculating the probability of selecting each set based on its $\tau$ and $\eta$, normalized by the previously calculated denominator. This gives the probability of choosing each set for inclusion based on its relative attractiveness and pheromone level.
    5. If the denominator is greater than 0 (indicating there are positive contributions from at least some sets), compute the probability as the weighted product of $\tau$ and $\eta$ divided by the denominator. If the denominator is 0 (indicating no heuristic or pheromone influence), the probability is set to 0 for all sets.
    6. Append each calculated probability to the list $P$.
    7. Return the list of probabilities $P$, representing the chance of each set being selected for the next step in the solution process.


- **`heuristicValue(self, ant)`**
    This method calculates the heuristic value $H$ for each set in the problem's universe ($S$), from the perspective of a given ant's current solution state. The heuristic value is a measure of the desirability or attractiveness of choosing a particular set, based on the current solution context. The process works as follows:

    1. Initialize an empty list $H$ to store the heuristic values for each set.
    2. Iterate through each set in $S$.
    3. Calculate the intersection of the current set ($S[i]$) with the `ant.uncovered` elements—those elements not yet covered by the ant's solution.
    4. If the intersection is non-empty (i.e., there are elements in the set that would cover currently uncovered elements), the heuristic value ($h_{val}$) for that set is calculated as the ratio of the number of elements in the intersection to the set's cost ($C[i]$). This ratio represents the efficiency of adding the set to the solution, balancing coverage gained against cost incurred.
    5. If the intersection is empty (indicating the set does not contribute to covering any uncovered elements), $h_{val}$ is set to 0, indicating no heuristic desirability.
    6. Append the calculated $h_{val}$ to the list $H$.
    7. Once all sets have been evaluated, return the list of heuristic values $H$.


**`eliminateRedundants(self, ant)`**

This function aims to refine an ant's solution in an optimization process, specifically by attempting to eliminate redundant elements (sets) from the solution. The process enhances solution efficiency by removing sets that do not contribute to covering any additional elements beyond those already covered by the remaining sets in the solution:

1. **Sort the Solution by Cost in Descending Order**: The ant's current solution (`ant.sol`) is sorted based on the cost of each set (`self.C[x]`), with the highest-cost sets placed first. This prioritization ensures that the attempt to remove sets starts with the most expensive ones, potentially leading to greater cost savings.

2. **Iterate Through the Sorted Solution**: For each set index (`i`) in the sorted solution (`sorted_sol`), the function performs the following steps:
    - **Create a Temporary Solution Without the Current Set**: A temporary solution (`temp_sol`) is generated by excluding the set `i` from the sorted solution. This effectively simulates the removal of the set to evaluate its impact.
    - **Instantiate a Temporary Ant with the Modified Solution**: A new temporary ant (`temp_ant`) is created with the temporary solution (`temp_sol`). This step allows evaluating how well the remaining sets cover the required elements without the excluded set.
    - **Compare Covered Elements**: The function compares the set of uncovered elements for both the current ant and the temporary ant. If the set of uncovered elements (`ant.uncovered`) remains unchanged when `i` is removed, it indicates that the set `i` is redundant—its removal does not lead to any new uncovered elements so it is removed from the original ant's solution (`ant.sol`).

3. **Update the Uncovered Elements** and Recalculate the Solution Cost**

$***$ ***Challenge*** $***$

It’s crucial to sort the solution initially because our goal at this stage is to refine our solution by eliminating redundant sets *without* losing any essential information. By organizing the solution in such a way that we start with the highest-cost sets, we ensure that when there’s a choice to be made between removing sets, we retain the lower-cost options in the solution. This strategic sorting is fundamental because it prioritizes the preservation of more cost-efficient sets, thereby optimizing our solution’s overall cost-effectiveness without compromising its coverage or completeness.

**`updatePheromoneIB(self, ant)`**

The function begins by iterating through the pheromone trail array `self.tau`, where each element represents the pheromone strength on a path or a component. It applies an evaporation process to all the pheromone values. Then it adds pheromone to sets that are part of the iteration's best solution, `self.it_best_sol`. For every component `i` in the ant's solution (`ant.sol`) that also appears in the iteration's best solution, the pheromone on that component is increased by an amount inversely proportional to the iteration's best cost `self.it_best_cost`. This reinforcement strategy promotes the selection of components that led to more cost-effective solutions in future iterations of the algorithm.

After applying evaporation and reinforcement, the function ensures that all pheromone values remain within specified bounds: `self.tau_min` and `self.tau_max`. This mechanism prevents the pheromone values from either becoming too low, which could make it hard for the algorithm to exploit learned solutions, or too high, which could make it hard for the algorithm to explore new paths. By maintaining pheromone strengths within these bounds, the algorithm maintains a balance between exploitation of known good solutions and the exploration of new potential solutions.

$***$ ***Challenge*** $***$

The parameter $\rho$ represents the rate of pheromone evaporation, a crucial parameter in MMAS algorithms that prevents the unlimited accumulation of pheromone strength which could, in turn, lead to premature convergence on suboptimal solutions. Therefore, tuning this value was an important and effective task:

- Too High Evaporation Rate: This means that the pheromones on the sets disappear quickly. While this can prevent the algorithm from prematurely converging on suboptimal solutions by encouraging exploration of new paths, it can also lead to a lack of focus. The algorithm didn't seem to exploit the best solutions found sufficiently, as the guiding pheromone trails evaporate too rapidly.

- Too Low Evaporation Rate: This means that pheromones remain on the paths for a longer time, leading to a higher likelihood of premature convergence to suboptimal solutions. The algorithm quickly became biased towards certain sets chosen early in the search process so the exploration of potentially better solutions was reduced.

**`localSearch(self, ant)`**

This function operates on the principle of local optimization, refining the ant's current solution by attempting to replace higher-cost elements with more cost-efficient alternatives:

1. **Sort Solution by Cost**: The ant's current solution set (`ant.sol`) is sorted in descending order based on the cost (`C[x]`) of each element in the solution. This sorting ensures the most expensive elements are positioned at the beginning of the list.

2. **Identify High-Cost Threshold**: The function selects the first element (`Q`) of the sorted solution - the one with the maximum cost. It then calculates a cost threshold (`E`) by multiplying the cost of `Q` by a factor $\rho_2$ given to the funciton. This threshold represents a cost limit for considering whether other sets can replace the high-cost sets in the solution.

3. **Prune Solution**: The ant's solution is pruned to retain only a certain percentage of the original solution. This is accomplished by discarding a portion of the solution starting from the most expensive element, based on a factor $\rho_1$. The remaining elements form a potentially more efficient base for rebuilding and refining the solution.

4. **Repopulate Solution**: The function iterates over all sets (`S`) in the problem domain. For each set not already in the solution (`ant.sol`) and with a cost (`C[i]`) less than or equal to the threshold (`E`), it adds the set to a list of potential candidates (`cols_to_select`). 

5. **Select Cost-Efficient Sets**: From the list of potential candidates, the function selects the set with the highest efficiency—defined as the lowest cost per element in the set (`C[x]/len(self.S[x])`). This set is then added to the ant's solution, and the elements it covers are removed from the set of uncovered elements (`ant.uncovered`).

6. **Repeat Until All Elements Covered**: This repopulation and optimization process repeats in a loop until there are no more uncovered elements in the problem domain (`ant.uncovered` is empty), ensuring the solution becomes as cost-efficient as possible while still covering all necessary elements.

$***$ ***Challenge*** $***$

The primary challenge encountered with this local search algorithm involved finely tuning the parameters $\rho_1$ and $\rho_2$ to achieve a balanced outcome. The goal was to ensure an effective number of elements were removed from the solution, making room for the inclusion of better alternatives. A higher value of $\rho_1$ resulted in a longer execution time for the algorithm, while setting it too low compromised its efficacy. Similarly, an excessively low setting for $\rho_2$ risked leaving the solution stagnant with no replacements made, whereas too high a value for $\rho_2$ proved inefficient and failed to contribute to the optimization process, which was the central objective of this function. By opting for the sets with the minimum cost among the potential candidates, I ensured the addition of cost-effective sets to the solution, thereby improving or at the very least not exacerbating the cost quality of the solution.

The provided functions appear to be part of the Min-Max Ant System (MMAS), one of the variations of the Ant Colony Optimization (ACO) algorithms, adapted to solve the Set Covering Problem. Let's describe what each function does:

**`updateIterationOptimum(self, ant)`**

This function is responsible for updating the best solution found in the current iteration:

First, the cost of the ant's solution is calculated. The algorithm then checks if the cost of the current ant's solution (`ant.cost`) is lower than the best solution cost found in the current iteration (`self.it_best_cost`). If true, this means the current ant's solution is better than any solution found so far in this iteration. In the end, the function calls `self.updateTauMM()`, signaling an update to the pheromone limits ($\tau_{max}, \tau_{min}$) based on the new iteration's best solution.

- **`updateTauMM(self)`**

    This function updates the pheromone limits used in the Min-Max Ant System. The updating is inspired by the equations provided in the aformentioned paper.
    For `self.tau_max`, It sets a new maximum pheromone level (`tau_max`) based on the best cost found in the current iteration (`self.it_best_cost`) inversely scaled by the evaporation rate (`self.rho`). This relationship ensures that better solutions (lower costs) result in higher pheromone limits, guiding future ant generations towards exploring similar solutions.

    The average `avg` represents an estimation of the expected size of a solution, assuming a uniform distribution of the qualities of solutions. It's calculated as half the size of the problem set `S`. `p` computes the probability based on the desired probability `self.p_best` that an ant chooses the best path under the assumption of only two choices, where `self.p_best` indicates the desired fraction of ants following the best path. It’s adjusted for the size of the set `S`.

    For `self.tau_min`, It calculates a new minimum pheromone level ($\tau_{min}$). This calculation is designed to ensure a minimum level of pheromone that is useful for guiding exploration by preventing the pheromone trail on any path from becoming so low that it is effectively ignored. The formula is engineered to balance between too much and too little pheromone reinforcement, allowing for both exploration of new paths and exploitation of known good paths.

In [172]:
class MMAS:
    def __init__(self, main_C, main_U, main_S, num_ants, num_iterations, alpha, beta, rho, tau_max, tau_min, p_best, nc_max, rho1, rho2):        
        self.num_ants = num_ants
        self.ants = []
        self.num_iterations = num_iterations

        self.C, self.U, self.S = main_C, main_U, main_S
        
        self.nc_max = nc_max #max rate of columns to uniformely choose (Diversification)

        self.rho1 = rho1 #column suppression rate (Local Search)
        self.rho2 = rho2 #reduction rate of the cost MAX (Local Search)

        self.tau_max = tau_max #phero max bound
        self.tau_min = tau_min #phero min bound
        self.p_best = p_best #prob that the sol only contains highest pheromone values
        self.alpha = alpha #power of pheromone trail
        self.beta = beta #power of heuristic information
        self.rho = rho #evaporation rate
        self.tau = np.array([self.tau_max for _ in range(len(self.S))]) #phero trails
        
        self.it_best_cost = np.inf
        self.it_best_sol = None
        self.best_ant = None
        self.it_best_cost_history = []
        self.global_best_cost = np.inf
        self.global_best_sol = None
    
    def runMMAS(self):
        for i in range(self.num_iterations):
            for j in range(self.num_ants):
                uniform_partial = self.diversification()
                ant = Ant(universe=self.U, sets=self.S, costs=self.C, solution=uniform_partial)
                self.ants.append(ant)
                ant = self.constructSolution(ant)
                self.eliminateRedundants(ant)               
                self.updateIterationOptimum(ant)
                self.updatePheromoneIB(ant)

            ant = self.localSearch(self.best_ant)
            self.updateIterationOptimum(ant)
            self.updatePheromoneIB(ant)
        return self.it_best_cost, self.it_best_sol
    
    def diversification(self):
        k = math.floor(self.nc_max * len(self.S))
        return random.sample(range(len(self.S[:len(self.S)//4])), k)
    
    def constructSolution(self, ant):
        while ant.uncovered:
            prob = self.calcProbability(ant)
            col_idx = np.random.choice(len(prob), p=prob)
            ant.sol.append(col_idx)
            ant.uncovered -= set(self.S[col_idx]) #update uncovered elements of the ant
            ant.cost += self.C[col_idx]
        return ant
    
    def heuristicValue(self, ant):
        H = []
        for i in range(len(self.S)):
            elem = len(set(self.S[i]) & ant.uncovered)
            if elem > 0:
                h_val = elem / self.C[i]
            else:
                h_val = 0
            H.append(h_val)
        return H
    
    def calcProbability(self, ant):
        heuristics = self.heuristicValue(ant)
        denom = sum((tau**self.alpha * eta**self.beta 
                                for tau, eta in zip(self.tau, heuristics)))
        P = [(tau**self.alpha * eta**self.beta) / denom 
                        if denom > 0 else 0
                        for tau, eta in zip(self.tau, heuristics)]
        return P

    def eliminateRedundants(self, ant):
        sorted_sol = sorted(ant.sol, key=lambda x: self.C[x], reverse=True)
        for i in sorted_sol:
            temp_sol = [x for x in sorted_sol if x != i]
            temp_ant = Ant(universe=self.U, sets=self.S, costs=self.C, solution=temp_sol)
            if ant.uncovered == temp_ant.uncovered:
                ant.sol.remove(i)
        ant.setUncovered()
        ant.calculateCost()
        return
    # update tau_max and tau_min values
    def updateTauMM(self):
        self.tau_max = 1.0 / (self.it_best_cost * self.rho)
        avg = len(self.S) / 2.0
        p = pow(self.p_best, 1.0 / len(self.S))
        self.tau_min = min(self.tau_max, self.tau_max * (1 - p) / ((avg - 1) * p))

    def updatePheromoneIB(self, ant):
        for i in range(len(self.tau)):
            self.tau[i] *= (1 - self.rho)
        
        for i in ant.sol:
            if i in self.it_best_sol:
                self.tau[i] += 1.0 / self.it_best_cost
        
        self.tau[self.tau < self.tau_min] = self.tau_min
        self.tau[self.tau > self.tau_max] = self.tau_max
        
    def localSearch(self, ant):
        sorted_sol = sorted(ant.sol, key=lambda x: self.C[x], reverse=True)
        Q = sorted_sol[0] #max cost of sol elements
        E = self.rho2 * self.C[Q]

        ant.sol = sorted_sol[int(math.floor(self.rho1 * len(ant.sol))):]
        ant.setUncovered()
        
        while ant.uncovered:
            cols_to_select = []
            for i in range(len(self.S)):
                if i not in ant.sol and self.C[i] <= E:
                    cols_to_select.append(i)
            if cols_to_select:
                selected = min(cols_to_select, key=lambda x: self.C[x]/len(self.S[x]))
                ant.sol.append(selected) 
                ant.uncovered -= set(self.S[selected])
        
        ant.setUncovered()
        ant.calculateCost()
        return ant
    
    def updateIterationOptimum(self, ant):
        ant.calculateCost()
        if ant.cost < self.it_best_cost:
            self.it_best_cost = ant.cost
            self.it_best_sol = ant.sol
            self.best_ant = ant
            self.updateTauMM()
            
    def updateGlobalOptimum(self):
        if self.it_best_cost < self.global_best_cost:
            self.global_best_cost = self.it_best_cost
            self.global_best_sol = self.it_best_sol
        self.it_best_cost = np.inf
        self.it_best_sol = None

## scp41.txt

In [150]:
costs_per_run = 0
main_C, main_U, main_S = getInput('scp41.txt')
for i in range(10):
    random.seed(i)
    np.random.seed(i)
    alg = MMAS(main_C, main_U, main_S,
           num_ants=15, 
           num_iterations=30, 
           alpha=8.0, beta=6.0, rho=0.5, tau_max=10.0, tau_min=0.1, p_best=0.05,
           nc_max=0.005,
           rho1=0.2, rho2=1.0) 
    C, S = alg.runMMAS()
    print(f'Best Cost (run {i+1})', C)
    print(f'Best Solution (run {i+1})', S)
    costs_per_run += C
    
print('Mean Cost', costs_per_run/10)

Length of 1D array Cost: 1000
Length of 1D array U (universe): 200
Length of 2D array S: 1000
Best Cost (run 1) 428
Best Solution (run 1) [105, 0, 1, 4, 2, 8, 13, 12, 10, 27, 9, 5, 25, 43, 19, 22, 34, 38, 80, 24, 45, 76, 11, 42, 7, 20, 60, 33, 90, 58, 56, 59, 31, 77, 67, 88, 68, 53, 47, 51, 17, 65, 61, 85, 48, 143, 115, 119, 49, 102, 111, 89, 274, 84, 147, 114, 74, 82, 137, 106, 120, 193, 123, 339, 127, 142]
Best Cost (run 2) 434
Best Solution (run 2) [88, 2, 0, 1, 13, 8, 42, 12, 14, 11, 25, 5, 10, 9, 4, 17, 27, 22, 19, 33, 18, 43, 20, 24, 16, 45, 60, 7, 46, 76, 65, 56, 53, 80, 90, 31, 58, 67, 47, 59, 97, 51, 85, 106, 77, 119, 137, 182, 114, 115, 123, 109, 82, 126, 102, 432, 68, 150, 192, 134, 101, 89, 120, 193, 35, 28, 127]
Best Cost (run 3) 427
Best Solution (run 3) [22, 0, 1, 2, 13, 12, 10, 5, 8, 19, 14, 18, 7, 27, 17, 34, 24, 31, 45, 76, 11, 48, 25, 20, 105, 46, 43, 42, 53, 28, 60, 88, 119, 33, 51, 65, 67, 59, 47, 90, 115, 143, 80, 58, 77, 49, 93, 68, 106, 107, 57, 56, 127, 72, 85,

## scp51.txt

In [117]:
costs_per_run = 0
main_C, main_U, main_S = getInput('scp51.txt')
for i in range(10):
    random.seed(i)
    np.random.seed(i)
    alg = MMAS(main_C, main_U, main_S,
           num_ants=15, 
           num_iterations=30, 
           alpha=8.0, beta=6.0, rho=0.5, tau_max=10.0, tau_min=0.1, p_best=0.05,
           nc_max=0.001,
           rho1=0.15, rho2=1.1) 
    C, S = alg.runMMAS()
    print(f'Best Cost (run {i+1})', C)
    print(f'Best Solution (run {i+1})', S)
    costs_per_run += C
    
print('Mean Cost', costs_per_run/10)


Length of 1D array Cost: 2000
Length of 1D array U (universe): 200
Length of 2D array S: 2000
Best Cost (run 1) 260
Best Solution (run 1) [0, 2, 12, 1, 3, 6, 4, 9, 42, 31, 13, 41, 30, 73, 11, 23, 86, 44, 87, 24, 27, 32, 28, 20, 64, 26, 35, 47, 17, 34, 45, 25, 74, 65, 33, 18, 143, 19, 38, 104, 142, 68, 56, 76, 36, 160, 60, 54, 43, 71, 92, 178, 80, 79, 194, 136, 89, 126, 189, 50, 90, 261, 159, 133, 154, 91, 5, 129]
Best Cost (run 2) 269
Best Solution (run 2) [93, 2, 1, 0, 12, 6, 87, 3, 4, 13, 28, 10, 7, 22, 24, 19, 17, 27, 44, 30, 26, 41, 23, 43, 78, 86, 42, 75, 91, 76, 84, 68, 73, 33, 31, 49, 159, 36, 46, 172, 38, 35, 56, 129, 89, 74, 18, 50, 51, 90, 80, 60, 71, 79, 137, 124, 194, 261, 104, 126, 234, 133, 154, 136, 92, 290, 5]
Best Cost (run 3) 255
Best Solution (run 3) [1, 6, 12, 2, 4, 3, 13, 7, 0, 10, 11, 19, 5, 24, 27, 41, 30, 17, 22, 18, 47, 23, 28, 31, 44, 46, 43, 36, 42, 25, 75, 33, 68, 86, 84, 124, 79, 29, 49, 51, 64, 76, 90, 62, 56, 60, 35, 137, 77, 159, 175, 314, 74, 80, 57, 71

## scp54.txt

In [94]:
costs_per_run = 0
main_C, main_U, main_S = getInput('scp54.txt')
for i in range(10):
    random.seed(i)
    np.random.seed(i)
    alg = MMAS(main_C, main_U, main_S,
           num_ants=15, 
           num_iterations=30, 
           alpha=8.0, beta=6.0, rho=0.4, tau_max=10.0, tau_min=0.1, p_best=0.05,
           nc_max=0.001,
           rho1=0.15, rho2=1.1) 
    C, S = alg.runMMAS()
    print(f'Best Cost (run {i+1})', C)
    print(f'Best Solution (run {i+1})', S)
    costs_per_run += C
    
print('Mean Cost', costs_per_run/10)



Length of 1D array Cost: 2000
Length of 1D array U (universe): 200
Length of 2D array S: 2000
Best Cost (run 1) 245
Best Solution (run 1) [1, 5, 7, 0, 26, 2, 11, 3, 13, 15, 4, 53, 17, 30, 27, 49, 48, 8, 33, 97, 50, 16, 32, 28, 38, 29, 59, 36, 37, 55, 6, 41, 68, 105, 64, 104, 102, 76, 126, 72, 44, 62, 31, 115, 39, 89, 147, 57, 85, 54, 165, 148, 237, 188, 207, 86, 82, 253, 108, 195, 128, 136, 293, 116, 156]
Best Cost (run 2) 246
Best Solution (run 2) [30, 17, 2, 11, 7, 5, 1, 0, 6, 4, 8, 3, 18, 50, 102, 32, 49, 13, 16, 14, 33, 48, 99, 38, 104, 108, 55, 21, 28, 36, 31, 41, 75, 53, 105, 116, 54, 76, 126, 97, 47, 211, 64, 39, 115, 147, 148, 62, 307, 57, 237, 129, 193, 195, 188, 165, 137, 87, 82, 207, 179, 29, 74]
Best Cost (run 3) 246
Best Solution (run 3) [0, 3, 5, 2, 49, 11, 6, 16, 18, 7, 28, 48, 26, 17, 21, 29, 1, 27, 30, 8, 99, 33, 50, 36, 53, 38, 37, 59, 108, 55, 104, 32, 68, 102, 41, 82, 76, 75, 31, 39, 64, 96, 207, 105, 115, 147, 126, 13, 86, 54, 165, 87, 195, 148, 85, 237, 62, 77, 18

## scpa2.txt

In [173]:
costs_per_run = 0
main_C, main_U, main_S = getInput('scpa2.txt')
for i in range(10):
    random.seed(i)
    np.random.seed(i)
    alg = MMAS(main_C, main_U, main_S,
           num_ants=15, 
           num_iterations=30, 
           alpha=8.0, beta=6.0, rho=0.4, tau_max=10.0, tau_min=0.1, p_best=0.05,
           nc_max=0.001,
           rho1=0.1, rho2=1.2) 
    C, S = alg.runMMAS()
    print(f'Best Cost (run {i+1})', C)
    print(f'Best Solution (run {i+1})', S)
    costs_per_run += C
    
print('Mean Cost', costs_per_run/10)

Length of 1D array Cost: 3000
Length of 1D array U (universe): 300
Length of 2D array S: 3000
Best Cost (run 1) 257
Best Solution (run 1) [0, 1, 14, 2, 4, 5, 27, 13, 6, 3, 59, 7, 12, 8, 24, 22, 15, 49, 30, 69, 52, 41, 87, 64, 21, 39, 37, 26, 91, 20, 90, 32, 25, 77, 48, 88, 80, 147, 102, 141, 51, 62, 202, 114, 101, 203, 158, 119, 118, 65, 40, 60, 126, 105, 144, 173, 122, 66, 85, 123, 94, 273, 180, 256, 197, 129, 116, 104, 142, 67, 355, 11, 74]
Best Cost (run 2) 252
Best Solution (run 2) [1, 0, 7, 24, 4, 8, 3, 5, 13, 27, 14, 12, 2, 61, 22, 9, 64, 31, 18, 30, 32, 17, 46, 57, 36, 37, 90, 21, 59, 41, 19, 52, 78, 79, 153, 39, 88, 69, 116, 51, 179, 102, 40, 77, 15, 114, 142, 141, 147, 91, 119, 118, 144, 203, 49, 244, 123, 176, 26, 130, 202, 247, 104, 74, 137, 129, 80, 100, 273, 105, 359, 11, 66]
Best Cost (run 3) 260
Best Solution (run 3) [0, 1, 2, 11, 14, 4, 18, 8, 26, 3, 22, 7, 27, 29, 31, 36, 13, 24, 37, 66, 30, 21, 15, 180, 41, 59, 45, 39, 64, 88, 87, 48, 52, 5, 32, 78, 90, 57, 62, 69, 14

## scpb1.txt

In [174]:
costs_per_run = 0
main_C, main_U, main_S = getInput('scpb1.txt')
for i in range(10):
    random.seed(i)
    np.random.seed(i)
    alg = MMAS(main_C, main_U, main_S,
           num_ants=15, 
           num_iterations=30, 
           alpha=8.0, beta=6.0, rho=0.4, tau_max=10.0, tau_min=0.1, p_best=0.05,
           nc_max=0.001,
           rho1=0.15, rho2=1.1) 
    C, S = alg.runMMAS()
    print(f'Best Cost (run {i+1})', C)
    print(f'Best Solution (run {i+1})', S)
    costs_per_run += C
    
print('Mean Cost', costs_per_run/10)


Length of 1D array Cost: 3000
Length of 1D array U (universe): 300
Length of 2D array S: 3000
Best Cost (run 1) 66
Best Solution (run 1) [1, 0, 15, 4, 7, 6, 9, 26, 5, 2, 10, 3, 20, 8, 14, 38, 37, 25, 18, 34, 30, 100, 24, 66, 27, 36, 31, 32, 49, 33, 45, 35, 29, 47, 126, 140, 11, 44, 17]
Best Cost (run 2) 70
Best Solution (run 2) [29, 15, 2, 13, 4, 6, 3, 7, 20, 21, 8, 9, 25, 26, 11, 37, 18, 30, 34, 31, 65, 33, 14, 38, 32, 66, 83, 126, 100, 101, 78, 93, 64, 1, 73, 54]
Best Cost (run 3) 65
Best Solution (run 3) [0, 2, 3, 10, 4, 14, 15, 7, 6, 8, 21, 30, 27, 9, 18, 20, 36, 24, 38, 37, 52, 49, 32, 31, 53, 144, 43, 29, 61, 125, 140, 26, 33, 25, 35, 65]
Best Cost (run 4) 68
Best Solution (run 4) [3, 1, 4, 0, 7, 6, 14, 20, 10, 37, 8, 38, 9, 21, 23, 27, 30, 15, 24, 25, 49, 66, 31, 100, 32, 33, 34, 26, 29, 77, 83, 144, 140, 35, 43, 18, 2, 5, 53]
Best Cost (run 5) 64
Best Solution (run 5) [3, 7, 5, 6, 4, 15, 0, 10, 9, 11, 41, 39, 29, 37, 18, 25, 21, 27, 32, 38, 49, 33, 66, 53, 45, 52, 140, 126, 94,